In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from openpyxl import load_workbook

dir1='bimodal genes'
dir2='genes normalization'
file_path='Bimodal Gene Info.xlsx'
filelist1=['Danish', 'Spanish',  'American','3_countries_combined' ,'Human_nose', 'Human_cheek', 'Human_forehead', 'Pig', 'Mouse', 'Marine']

# Load the workbook
wb = load_workbook(file_path)
# Select the worksheet to operate on
sheet = wb['Sheet1']  
# Define the starting row and column numbers to write
rows = 2
cols = 3

for k in range(len(filelist1)):
    df1=pd.read_csv(dir1+'/'+filelist1[k]+'_BiDgenes',sep='\t')
    df2=pd.read_csv(dir2+'/'+filelist1[k],sep='\t',index_col=0)
    df2T=df2.T
    Number_of_samples = len(df2T.index)
    Number_of_genes = len(df2T.columns)
    bimodal_genes = df1.iloc[:,0].to_list()
    Number_of_bimodal_genes = len(bimodal_genes)

    #Calculate the boundary_valley value of the bimodal distribution.
    boundary_valley = []
    for i in range(len(bimodal_genes)):
            y=df2T[bimodal_genes[i]]
            if np.sum(abs(np.log(y[y>0]))>0)>0.3*len(df2T): 
                data = np.log(y[y>0]) 
                kde = gaussian_kde(data) 
                kde_data = kde.evaluate(np.linspace(min(data), max(data), 1000))  
                kde_x=np.linspace(min(data), max(data), 1000)
                kde_y=kde(kde_x) 
                dy_dx = np.gradient(kde_y, kde_x) 
                peakposi_y=[] 
                valeyposi_y=[] 
                peakposi_x=[] 
                valeyposi_x=[] 
                for j in range(len(dy_dx)):
                    if dy_dx[j-1]>=0 and dy_dx[j]<=0 : 
                        peak_y=kde_y[j-1]
                        peakposi_y.append(peak_y)
                        peak_x=kde_x[j-1]
                        peakposi_x.append(peak_x)
                    if dy_dx[j-1]<=0 and dy_dx[j]>=0:
                        valey_y=kde_y[j]
                        valeyposi_y.append(valey_y)
                        valey_x=kde_x[j]
                        valeyposi_x.append(valey_x)
                if len(peakposi_y)>=2: 
                    Ymax1=max(peakposi_y) 
                    BB=[]
                    for a in peakposi_y:
                        BB.append(a)
                    BB.remove(max(BB))
                    Ymax2=max(BB) 
                    Xmax1=peakposi_x[peakposi_y.index(Ymax1)] 
                    Xmax2=peakposi_x[peakposi_y.index(Ymax2)] 
                    Yvaley=[] 
                    for b in valeyposi_x:
                        if Xmax1<b<=Xmax2 or Xmax2<b<=Xmax1:
                            CC=valeyposi_y[valeyposi_x.index(b)]
                            Yvaley.append(CC)
                    x=np.exp(valeyposi_x[(valeyposi_y.index(min(Yvaley)))])
                    boundary_valley.append(x)
    if len(bimodal_genes)==len(boundary_valley):
        maxtrix=np.full((len(df2T),len(bimodal_genes)),2, dtype=object) 
        for i in range(len(boundary_valley)):
            for j in range(len(df2T)):
                math1=df2T[bimodal_genes[i]].tolist()[j]
                math2=boundary_valley[i]
                if math1<math2 :
                    maxtrix[j,i]=0
                if math1==0:
                    maxtrix[j,i]=None
                elif math1>=math2:
                    maxtrix[j,i]=1     
    else: print(False)
    #Combine the 0s and 1s in the matrix row by row into a binary number，and convert the binary number to a decimal number.
    Grouping_list=[]
    for i in range(len(maxtrix)):
        a = maxtrix[i,:]
        a = [0 if x is None else x for x in a]
        a = [str(i) for i in a] # Convert each element to a string
        b = int(''.join(a),2) # Convert binary string to decimal integer
        Grouping_list.append(b)
    #calculate the number of unique values in the Grouping_list
    Number_of_binary_functional_types=len(set(Grouping_list))

    sheet.cell(row=rows+k, column=cols).value = Number_of_samples
    sheet.cell(row=rows+k, column=cols+1).value = Number_of_genes
    sheet.cell(row=rows+k, column=cols+2).value = Number_of_bimodal_genes
    # Convert the list to a comma-separated string
    Bimodal_genes_str = ', '.join(bimodal_genes)
    sheet.cell(row=rows+k, column=cols+3).value = Bimodal_genes_str
    sheet.cell(row=rows+k, column=cols+4).value = Number_of_binary_functional_types

    wb.save(file_path)
    print(f'Written to row:{rows+k}, {filelist1[k]}')

Written to row:2, Danish
Written to row:3, Spanish
Written to row:4, American
Written to row:5, 3_countries_combined
Written to row:6, Human_nose
Written to row:7, Human_cheek
Written to row:8, Human_forehead
Written to row:9, Pig
Written to row:10, Mouse
Written to row:11, Marine
